In [1]:
# flashalgorithm is my package that is available on github
import flashalgorithm as fc
import numpy as np
import pickle

comp_list = ('water', 'methane', 'ethane', 'propane')
phase_list = ('aqueous', 'vapor', 'lhc', 's1', 's2')
P = 165.0  # bar
T = 289.0 # Kelvin

flash_full = fc.FlashController(components=comp_list,
                                phases=phase_list)

water_fracs = [0.7, 0.92]
hc_fracs = np.linspace(0, 1, 20)
c1_frac, c2_frac, c3_frac = np.meshgrid(hc_fracs, hc_fracs, hc_fracs) #Generically for three hydrocarbons


z_all = list()
for water in water_fracs:
    for hcs in zip(c1_frac.flatten(), c2_frac.flatten(), c3_frac.flatten()):
        if sum(hcs) > 0.0:
            mod_hcs = [x / sum(hcs) * (1.0 - water) for x in hcs]
            z = np.asarray([water] + mod_hcs)
            z_all.append(z / np.sum(z))
z_use = np.unique(np.asarray(z_all), axis=0)


def emptycomp_hash(z):
    hashed = sum([2**ii for ii, x in enumerate(z) if x == 0.0])
    return hashed

np.take(z_use,np.random.permutation(z_use.shape[0]),axis=0,out=z_use)
flash_dict = {0: flash_full}
all_output = list()
out_file = 'flashtable_{0}bar{1}K.pkl'.format(P,T)
K_use = []

# For generating flash table without propane
no_propane = z_use[:,-1] == 0
z_use = z_use[no_propane, :]


# Main loop
for ii, z in enumerate(z_use):
    comp_hash = emptycomp_hash(z)

    new_comps, new_z = zip(*[
        (comp, z_) for comp, z_ in zip(comp_list, z)
        if z_ != 0.0
    ])
    if comp_hash not in flash_dict.keys():
        flash_dict.update({comp_hash:
                               fc.FlashController(
                                    components=new_comps,
                                    phases=phase_list)})
    flash_use = flash_dict[comp_hash]
    new_z = np.asarray(new_z)
    try:
        output = flash_use.main_handler(
                    compobjs=flash_use.compobjs,
                    z=new_z,
                    T=T,
                    P=P,
                    initialize=True)
        if output[-1] > 1e-6:
            output = flash_use.main_handler(
                compobjs=flash_use.compobjs,
                z=new_z,
                T=T,
                P=P,
                initialize=True,
                incipient_calc=True)
    except:
        try:
            output = flash_use.main_handler(
                        compobjs=flash_use.compobjs,
                        z=new_z,
                        T=T,
                        P=P,
                        initialize=True,
                        incipient_calc=False)
            if output[-1] > 1e-6:
                output = flash_use.main_handler(
                    compobjs=flash_use.compobjs,
                    z=new_z,
                    T=T,
                    P=P,
                    initialize=True,
                    incipient_calc=True)
        except:
            try:
                if output[-1] > 1e-6:
                    output = flash_use.main_handler(
                        compobjs=flash_use.compobjs,
                        z=new_z,
                        T=T,
                        P=P,
                        initialize=True,
                        incipient_calc=True)
            except:
                output = []


    all_output.append([ii, z, new_comps, new_z, output])
    if np.mod(ii, 20) == 0:
        print('{0:3.3f} % complete!'.format(float(ii) * 100 / len(z_use)))
        with open(out_file, 'wb') as f:
            pickle.dump(all_output, f)

print('{0:3.3f} % complete!'.format(float(ii) * 100 / len(z_use)))
with open(out_file, 'wb') as f:
     pickle.dump(all_output, f)

0.000 % complete!
3.906 % complete!
7.812 % complete!
11.719 % complete!
15.625 % complete!
19.531 % complete!
23.438 % complete!
27.344 % complete!
31.250 % complete!
35.156 % complete!
39.062 % complete!
42.969 % complete!
46.875 % complete!
50.781 % complete!
54.688 % complete!
58.594 % complete!
62.500 % complete!
66.406 % complete!
70.312 % complete!
74.219 % complete!
78.125 % complete!
82.031 % complete!
85.938 % complete!
89.844 % complete!
93.750 % complete!
97.656 % complete!
99.805 % complete!
